## BYOL

[[arxiv] 🎓 Bootstrap Your Own Latent](https://arxiv.org/abs/2006.07733)

<center><img src ="https://ml.gan4x4.ru/msu/additions/L08/byol_performance.png"  width="400"></center>

<center><em>Source: <a href="https://arxiv.org/pdf/2006.07733">Bootstrap Your Own Latent</a></em></center>


Методика, описанная в статье, использует дистилляцию для получения качественного представления данных в пространстве признаков (latent space).

**Идеи метода**:

1. В интернете масса неразмеченных изображений, давайте будем всячески искажать (аугментировать) их: обрезать, масштабировать, поворачивать и затем подавать на вход сети.

Если искажения не меняют объект до неузнаваемости, то его класс поменяться не должен, следовательно, можно требовать от модели, чтобы вектора признаков у аугментированных изображений были похожи.

Так как меток у нас нет, то в качестве loss можно использовать расстояние между векторами признаков. Для векторов, полученных из одного изображения, оно должно быть меньше, чем между векторами различных изображений.

Это идея [Contrastive Loss 🎓[article]](https://paperswithcode.com/method/supervised-contrastive-loss).

При наивной реализации такой подход работает не слишком хорошо, так как общее количество сравнений для датасета с N изображениями и M аугментациями — это $ (N*M)^2$. Поэтому приходится выбирать из всех возможных пар только часть самых сложных ([Hard example Mining 🎓[article]](https://paperswithcode.com/method/ohem)).

2. Авторы статьи предложили решение, основанное на идее дистилляции.

Создаются две одинаковые сети (online и target) и два различных набора аугментаций (t и t`). В первую сеть подаются изображения, аугментированные при помощи t, во вторую — при помощи t'.

Предсказания моделей сравниваются, и loss подсчитывается как расстояние между предсказаниями, но обновление весов методом градиентного спуска происходит **только у одной модели** (online).

Веса второй (target) постепенно обновляются как [экспоненциальное скользящее среднее 📚[wiki]](https://en.wikipedia.org/wiki/Exponential_smoothing) от весов первой модели.

$\xi \leftarrow \tau \xi + (1-\tau\theta),$

где $\tau$ — скорость обновления, $\theta$ — веса другой (online) модели.

<center><img src ="https://ml.gan4x4.ru/msu/additions/L08/byol_scheme.png"  width="900"></center>

<center><em>Source: <a href="https://arxiv.org/pdf/2006.07733">Bootstrap Your Own Latent</a></em></center>

Затем достаточно получить embedding-и для изображений из ImageNet и классифицировать их при помощи линейного классификатора.


Большая библиотека, где реализован BYOL:
* [[doc] 🛠️ MMSelfSup](https://mmselfsup.readthedocs.io/en/latest/get_started.html#install-on-google-colab)
* [[git] 🐾 MMSelfSup](https://github.com/open-mmlab/mmselfsup)


Чтобы не качать зависимости, воспользуемся [этой реализацией 🐾[git]](
https://github.com/lucidrains/byol-pytorch).

In [ ]:
!pip install -q byol-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.8/917.8 kB 9.5 MB/s eta 0:00:00


In [ ]:
import torch
from byol_pytorch import BYOL
from torchvision import models
from tqdm import tqdm


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet50(weights=None)
learner = BYOL(resnet, image_size=256, hidden_layer="avgpool")

learner.to(device)
opt = torch.optim.Adam(learner.parameters(), lr=3e-4)


def sample_unlabelled_images():
    return torch.randn(20, 3, 256, 256)


for _ in tqdm(range(3)):
    images = sample_unlabelled_images()
    loss = learner(images.to(device))
    opt.zero_grad()
    loss.backward()
    opt.step()
    learner.update_moving_average()  # update moving average of target encoder

# save your improved network
torch.save(resnet.state_dict(), "./improved-net.pt")

100%|██████████| 3/3 [03:11<00:00, 63.83s/it]
